# PREPARACION PARA ADOBE

### Este documento busca la data , la relaciona y la sirve de acuerdo a las tablas y tipos de datos que solicitó el partner

In [1]:
import pandas as pd
from pymongo import MongoClient
import pymongo
from datetime import datetime, timedelta
import numpy as np

import csv


In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ecdf"]
mycol_users = mydb["users"]

In [3]:
feh=datetime(2017,1,1,0,0,0)

In [4]:
def userframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
userData = userframes(mycol_users.find({'createdAt': {'$gte':feh}},{'_id':1,'createdAt':1,'cliente_id':1 ,'email':1,'verificado': 1 }), 10000)


#### PROFILE TABLE

In [5]:
id_user=userData

In [6]:
id_user=id_user.rename(columns={ "createdAt": "REGISTRATION_DATE","verificado": "HAS_EMAIL_VERIFY",'email':'EMAIL'})

In [7]:
mydb = myclient["cdfMaster"]

feh=datetime(2017,1,1,0,0,0)

In [8]:
mycol_clientes = mydb["otroClientes"]
def clientesframes(iterator, chunk_size: int):
  """Turn an iterator into multiple small pandas.DataFrame
  This is a balance between memory and efficiency
  """
  records = []
  frames = []
  for i, record in enumerate(iterator):
    records.append(record)
    if i % chunk_size == chunk_size - 1:
      frames.append(pd.DataFrame(records))
      records = []
  if records:
    frames.append(pd.DataFrame(records))
  return pd.concat(frames) if frames else pd.DataFrame()
  
CLIENTES = clientesframes(mycol_clientes.find({
},{'fecha_nacimiento':1,'pais':1,'equipo_favorito':1 ,'nombre':1,'apellidoPaterno': 1,'telefono':1,'genero': 1  }), 10000)


In [9]:
CLIENTES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434256 entries, 0 to 4255
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   _id               434256 non-null  object
 1   nombre            433777 non-null  object
 2   apellidoPaterno   433885 non-null  object
 3   pais              150140 non-null  object
 4   fecha_nacimiento  350036 non-null  object
 5   genero            261321 non-null  object
 6   equipo_favorito   211979 non-null  object
 7   telefono          2535 non-null    object
dtypes: object(8)
memory usage: 29.8+ MB


#### Se le quiere añadir el id de usuario a la tabla de clientes

In [10]:
CLIENTES['cliente_id']=CLIENTES['_id']

In [11]:
CLIENTES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434256 entries, 0 to 4255
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   _id               434256 non-null  object
 1   nombre            433777 non-null  object
 2   apellidoPaterno   433885 non-null  object
 3   pais              150140 non-null  object
 4   fecha_nacimiento  350036 non-null  object
 5   genero            261321 non-null  object
 6   equipo_favorito   211979 non-null  object
 7   telefono          2535 non-null    object
 8   cliente_id        434256 non-null  object
dtypes: object(9)
memory usage: 33.1+ MB


In [12]:
consolidados2 = CLIENTES.merge(id_user, on='cliente_id', how='right', indicator=True)

In [13]:
consolidados2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 0 to 287394
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id_x              286767 non-null  object        
 1   nombre             286288 non-null  object        
 2   apellidoPaterno    286396 non-null  object        
 3   pais               2999 non-null    object        
 4   fecha_nacimiento   203019 non-null  object        
 5   genero             113949 non-null  object        
 6   equipo_favorito    193647 non-null  object        
 7   telefono           1803 non-null    object        
 8   cliente_id         287335 non-null  object        
 9   _id_y              287395 non-null  object        
 10  REGISTRATION_DATE  287395 non-null  datetime64[ns]
 11  EMAIL              287395 non-null  object        
 12  HAS_EMAIL_VERIFY   287394 non-null  object        
 13  _merge             287395 non-null  category

In [14]:
consolidados2['REGISTRATION_DATE'] = pd.to_datetime(consolidados2['REGISTRATION_DATE'])

In [15]:
consolidados2['REGISTRATION_DATE'] 

0        2017-01-01 01:32:15.159
1        2017-01-01 13:43:58.600
2        2017-01-01 15:30:53.612
3        2017-01-01 16:35:54.370
4        2017-01-01 16:45:30.095
                   ...          
287390   2020-04-25 14:49:56.616
287391   2020-04-25 15:02:47.214
287392   2020-04-25 15:03:26.562
287393   2020-04-25 15:03:45.277
287394   2020-04-25 17:00:20.071
Name: REGISTRATION_DATE, Length: 287395, dtype: datetime64[ns]

In [16]:

profile=consolidados2

TRABAJAR LAS FECHAS DE NACIMIENTO

In [17]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 0 to 287394
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _id_x              286767 non-null  object        
 1   nombre             286288 non-null  object        
 2   apellidoPaterno    286396 non-null  object        
 3   pais               2999 non-null    object        
 4   fecha_nacimiento   203019 non-null  object        
 5   genero             113949 non-null  object        
 6   equipo_favorito    193647 non-null  object        
 7   telefono           1803 non-null    object        
 8   cliente_id         287335 non-null  object        
 9   _id_y              287395 non-null  object        
 10  REGISTRATION_DATE  287395 non-null  datetime64[ns]
 11  EMAIL              287395 non-null  object        
 12  HAS_EMAIL_VERIFY   287394 non-null  object        
 13  _merge             287395 non-null  category

In [18]:
profile.loc[(profile['fecha_nacimiento'].str.contains("sin informacion|1900-01-01T04:42:46.000Z", na=False)),['fecha_nacimiento']]= np.NaN

In [19]:
profile['fecha_nacimiento']= pd.to_datetime(profile.fecha_nacimiento, utc=True)

In [20]:
profile.loc[(profile['fecha_nacimiento']<'1910-01-01T04:42:46.000Z'),['fecha_nacimiento']]= np.NaN

In [21]:
profile['fecha_nacimiento'] = pd.to_datetime(profile['fecha_nacimiento'], format='%d/%m/%y %H:%M:%S').dt.strftime('%Y/%m/%d')

In [22]:
profile=profile.rename(columns={ "cliente_id": "CUSTOMER_ID","fecha_nacimiento": "DATE_OF_BIRTH","pais": "COUNTRY","equipo_favorito": "FAVORITE_TEAM","nombre": "FIRST_NAME","apellidoPaterno":"LAST_NAME","telefono":"PHONE"})

In [23]:
profile=profile.drop(['_id_x', '_id_y','_merge'],axis=1)

In [24]:
profile['cusGenderNonBinary']=profile['genero']

#### Hay que transformar el sexo  para llenar el campo cusGenderNonBinary que debe ser numérico

In [25]:
profile.loc[(profile['cusGenderNonBinary'].str.contains("m|M|male", na=False)),['cusGenderNonBinary']]='male'

In [26]:
profile.loc[(profile['cusGenderNonBinary'].str.contains("f|F|female", na=False)),['cusGenderNonBinary']]='female'

#### Hay que transformar el sexo para llenar el campo cusGenderNonBinary que debe ser numérico

In [27]:
profile.loc[(profile['genero'].str.contains("m|M|male", na=False)),['genero']]='M'

In [28]:
profile.loc[(profile['genero'].str.contains("f|F|female", na=False)),['genero']]='F'

In [29]:
profile.loc[(profile['genero'].str.contains("None|nan", na=False)),['genero']]=''

In [30]:
profile=profile.rename(columns={ 'genero':'GENDER','cusGenderNonBinary':'GENDER_NB'})

In [31]:
profile = profile[['CUSTOMER_ID','COUNTRY','DATE_OF_BIRTH','EMAIL','FAVORITE_TEAM','FIRST_NAME','LAST_NAME','GENDER','GENDER_NB','PHONE','REGISTRATION_DATE','HAS_EMAIL_VERIFY']]

In [32]:
profile['CUSTOMER_ID'] = profile['CUSTOMER_ID']

In [33]:
profile['COUNTRY'] =profile['COUNTRY'] 

In [34]:
profile['FAVORITE_TEAM'] = profile['FAVORITE_TEAM']

In [35]:
profile['FIRST_NAME'] =profile['FIRST_NAME'] 

In [36]:
profile['LAST_NAME'] = profile['LAST_NAME']

In [37]:
profile['GENDER'] = profile['GENDER'] 

In [38]:
profile['GENDER_NB'] = profile['GENDER_NB']

In [39]:
profile['EMAIL'] = profile['EMAIL']

In [40]:
profile['REGISTRATION_DATE']=pd.to_datetime(profile['REGISTRATION_DATE'], format='%d/%m/%y %H:%M:%S').dt.strftime('%Y/%m/%d %H:%M:%S')

In [41]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287395 entries, 0 to 287394
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   CUSTOMER_ID        287335 non-null  object
 1   COUNTRY            2999 non-null    object
 2   DATE_OF_BIRTH      113308 non-null  object
 3   EMAIL              287395 non-null  object
 4   FAVORITE_TEAM      193647 non-null  object
 5   FIRST_NAME         286288 non-null  object
 6   LAST_NAME          286396 non-null  object
 7   GENDER             113949 non-null  object
 8   GENDER_NB          113949 non-null  object
 9   PHONE              1803 non-null    object
 10  REGISTRATION_DATE  287395 non-null  object
 11  HAS_EMAIL_VERIFY   287394 non-null  object
dtypes: object(12)
memory usage: 28.5+ MB


In [42]:
import datetime

 #### Ahora vamos a determinar el nombre y el path

In [43]:
name='cdf_profile_'+datetime.datetime.today().strftime('%Y%m%d_%H%M%S')

In [44]:
path='/home/tecnoboy/'+name+'.csv'

In [45]:
profile.to_csv(path, index=False, quoting=csv.QUOTE_NONNUMERIC)